<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/BERT_on_IEMOCAP_for_DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import re
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam  
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical  
import time

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from keras.applications.resnet50 import ResNet50

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
import pickle

In [3]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 22.1MB/s 


In [4]:
import tokenization

In [5]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [6]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = pooled_output
    clf_output1 = tf.reshape(clf_output,[32,32,3])
    base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape=clf_output1.shape)
    print("Hi")
    # x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    out = Dense(6, activation='softmax')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model,clf_output1

In [6]:
# base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (32,32,3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# predictions = Dense(6 , activation= 'softmax')(x)
# model = Model(inputs = base_model.input, outputs = predictions)

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 40.2 s, sys: 12.1 s, total: 52.3 s
Wall time: 2min 50s


In [9]:
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)

In [10]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [11]:
processed_data,processed_label = preprocessing(data,labels)
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [12]:
for i in range(0,len(processed_data)):
  processed_data[i]= ' '.join(processed_data[i])
for i in range(0,len(test_processed_data)):
  test_processed_data[i]=' '.join(test_processed_data[i])

In [13]:
processed_data_batching=processed_data
test_processed_data_batching=test_processed_data

In [14]:
processed_data=np.asarray(processed_data)
test_processed_data=np.asarray(test_processed_data)
Y=to_categorical(processed_label, num_classes=6)
test_Y=to_categorical(test_processed_label, num_classes=6)

In [15]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [17]:
t0 = time.time()
train_input = bert_encode(processed_data, tokenizer, max_len=160)
test_input = bert_encode(test_processed_data, tokenizer, max_len=160)
train_labels = Y
test_labels = test_Y

In [16]:
model,clf_output1 = build_model(bert_layer, max_len=161)
model.summary()

Hi
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 161)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 161)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 161)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]   

In [18]:
train_history = model.fit(
    train_input, train_labels,
    epochs=1,
    batch_size=1
)

model.save('model.h5')

4699/4699 [==============================] - 822s 175ms/step - loss: 1.5394 - accuracy: 0.3871


**SAVING THE INTERMEDIATE LAYER OUTPUT**

In [19]:
print(len(processed_data_batching))

4699


In [20]:
print(len(test_processed_data_batching))

1623


In [21]:
count=0
batch=[]
batch_processed_data=[]
for i in processed_data_batching:
  batch.append(i)
  count=count+1
  if count==50:
    batch_processed_data.append(batch)
    batch=[]
    count=0
batch_processed_data.append(batch)

In [22]:
count=0
batch=[]
test_batch_processed_data=[]
for i in test_processed_data_batching:
  batch.append(i)
  count=count+1
  if count==50:
    test_batch_processed_data.append(batch)
    batch=[]
    count=0
test_batch_processed_data.append(batch)

In [25]:
train_labels = Y
layer_names=[layer.name for layer in model.layers]
layer_name=layer_names[3]
intermediate_encodings=[]
for processed_data in batch_processed_data:
  train_input = bert_encode(processed_data, tokenizer, max_len=160)
  intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
  intermediate_output = intermediate_layer_model.predict(train_input)
  intermediate_encodings.append(intermediate_output)
print(len(intermediate_encodings))

94


In [26]:
test_labels = test_Y
layer_names=[layer.name for layer in model.layers]
layer_name=layer_names[3]
test_intermediate_encodings=[]
for processed_data in test_batch_processed_data:
  train_input = bert_encode(processed_data, tokenizer, max_len=160)
  intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
  intermediate_output = intermediate_layer_model.predict(train_input)
  test_intermediate_encodings.append(intermediate_output)
print(len(test_intermediate_encodings))

33


In [35]:
#Logic for creating the 32x32x3 shape
# intermediate_encodings_in_np = np.array(intermediate_encodings)
# inter_reshaped=intermediate_encodings_in_np[0][0][0].reshape(32,32)
# inter_expanded=np.repeat(inter_reshaped[..., np.newaxis],3,-1)

In [83]:
print(len(intermediate_encodings[92][0]))

50


In [27]:
resnet_input=[]
for i in range(0,93):
  for j in range(0,50):
    temp=np.array(intermediate_encodings[i][0][j])
    temp2=temp.reshape(32,32)
    temp3=np.repeat(temp2[...,np.newaxis],3,-1)
    resnet_input.append(list(temp3))
for i in range(0,49):
    temp=np.array(intermediate_encodings[93][0][i])
    temp2=temp.reshape(32,32)
    temp3=np.repeat(temp2[...,np.newaxis],3,-1)
    resnet_input.append(list(temp3))

In [40]:
test_resnet_input=[]
for i in range(0,32):
  for j in range(0,50):
    temp=np.array(test_intermediate_encodings[i][0][j])
    temp2=temp.reshape(32,32)
    temp3=np.repeat(temp2[...,np.newaxis],3,-1)
    test_resnet_input.append(list(temp3))
for i in range(0,23):
    temp=np.array(test_intermediate_encodings[32][0][i])
    temp2=temp.reshape(32,32)
    temp3=np.repeat(temp2[...,np.newaxis],3,-1)
    test_resnet_input.append(list(temp3))

In [28]:
model = ResNet50(include_top=False, input_shape=(32,32,3), weights='imagenet')

# Freeze all the layers
for layer in model.layers[:]:
    layer.trainable = False
output = model.output
output = GlobalAveragePooling2D()(output)
output = Dense(units=6, activation='softmax')(output)
model = Model(model.input, output)

94773248/94765736 [==============================] - 0s 0us/step


In [31]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(np.array(resnet_input), train_labels, epochs=5, batch_size=1)

Epoch 1/5
4699/4699 [==============================] - 36s 8ms/step - loss: 1.2906 - categorical_accuracy: 0.5227
Epoch 2/5
4699/4699 [==============================] - 37s 8ms/step - loss: 1.2765 - categorical_accuracy: 0.5207
Epoch 3/5
4699/4699 [==============================] - 37s 8ms/step - loss: 1.2671 - categorical_accuracy: 0.5350
Epoch 4/5
4699/4699 [==============================] - 37s 8ms/step - loss: 1.2762 - categorical_accuracy: 0.5271
Epoch 5/5
4699/4699 [==============================] - 36s 8ms/step - loss: 1.2683 - categorical_accuracy: 0.5337


In [42]:
score = model.evaluate(np.array(test_resnet_input), test_Y)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

51/51 [==============================] - 1s 13ms/step - loss: 1.6562 - categorical_accuracy: 0.4319
Test loss: 1.656169056892395
Test accuracy: 0.431916207075119


In [23]:
y_pred=model.predict(test_input, batch_size=1)

In [24]:
y_pred1=np.argmax(y_pred,axis=1)
test_processed_label=np.asarray(test_processed_label)
t1 = time.time()
total = t1-t0
print("Total Execution time (Training + Testing): " + str(total))
print("Accuracy : " + str(accuracy_score(test_processed_label, y_pred1)))
print("Weighted F1-score : " + str(f1_score(test_processed_label, y_pred1, average='weighted')))

Total Execution time (Training + Testing): 948.8990435600281
Accuracy : 0.5169439309919901
Weighted F1-score : 0.5111446336164024
